In [7]:
# NBVAL_SKIP
import os

config = {
    "logger": {
        "log_level": "DEBUG",
        "log_file_path": None,
        "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    },
    "data": {
        "name": "IllustrisAPI",
        "args": {
            "api_key": os.environ.get("ILLUSTRIS_API_KEY"),
            "particle_type": ["stars", "gas"],
            "simulation": "TNG50-1",
            "snapshot": 99,
            "save_data_path": "data",
        },
        
        "load_galaxy_args": {
        "id": 12,
        "reuse": True,
        },

        "subset": {
            "use_subset": True,
            "subset_size": 1000,
        },
    },
    "simulation": {
        "name": "IllustrisTNG",
        "args": {
            "path": "data/galaxy-id-12.hdf5",
        },
    },
    "output_path": "output",

        
}

Convert the Data into Rubix Galaxy HDF5. This will make the call to the IllustrisAPI to download the data, and then convert it into the rubix hdf5 format using the input handler

In [8]:
# NBVAL_SKIP
from rubix.core.data import convert_to_rubix

In [9]:
# NBVAL_SKIP
convert_to_rubix(config)

2024-07-12 15:26:34,969 - rubix - INFO - Loading data from IllustrisAPI
2024-07-12 15:26:34,971 - rubix - DEBUG - Loading galaxy with ID 12
2024-07-12 15:26:34,972 - rubix - DEBUG - Performing GET request from http://www.tng-project.org/api/TNG50-1/snapshots/99/subhalos/12/cutout.hdf5?stars=Coordinates,GFM_InitialMass,GFM_Metallicity,GFM_StellarFormationTime,Velocities&gas=Coordinates,Density,Masses,GFM_Metallicity,StarFormationRate,InternalEnergy,Velocities,ElectronAbundance, with parameters None
2024-07-12 15:26:37,186 - rubix - DEBUG - Performing GET request from http://www.tng-project.org/api/TNG50-1/snapshots/99/subhalos/12, with parameters None
2024-07-12 15:26:37,415 - rubix - DEBUG - Appending subhalo data for subhalo 12
2024-07-12 15:26:37,449 - rubix - INFO - Loading data into input handler
2024-07-12 15:26:37,450 - rubix - DEBUG - Loading data from Illustris file..
2024-07-12 15:26:37,451 - rubix - DEBUG - Checking if the fields are present in the file...
2024-07-12 15:26:37

'output'

In [11]:
from rubix.core.data import convert_to_rubix, prepare_input
import sys
import jax.numpy as jnp

rubixdata = prepare_input(config)
type(rubixdata)
sys.getsizeof(rubixdata)
len(rubixdata.stars.coords)
attr = [attr for attr in dir(rubixdata.stars) if not attr.startswith('__')]# and attr not in ('coords', 'velocity')]
print(attr)

rubixdata.stars.__getattribute__(attr[3])
#rubixdata.stars.__getattribute__(attr[0]) = jnp.array(rubixdata.stars.__getattribute__(attr[0]))
#dir(rubixdata.galaxy)
#print(rubixdata.stars.__getattribute__(attr[2]))
attribute_value = rubixdata.stars.__getattribute__(attr[2])  # Get the current value
jax_array_value = jnp.array(attribute_value)  # Convert it to a JAX array
setattr(rubixdata.stars, attr[2], jax_array_value)  # Set the converted value back
#print(rubixdata.stars.__getattribute__(attr[2]))  # Check that the value has been updated
print(type(rubixdata.stars.coords))

{'logger': {'log_level': 'DEBUG', 'log_file_path': None, 'format': '%(asctime)s - %(name)s - %(levelname)s - %(message)s'}, 'data': {'name': 'IllustrisAPI', 'args': {'api_key': '05add2d69d501eb62014faa4dadb1af0', 'particle_type': ['stars', 'gas'], 'simulation': 'TNG50-1', 'snapshot': 99, 'save_data_path': 'data'}, 'load_galaxy_args': {'id': 12, 'reuse': True}, 'subset': {'use_subset': True, 'subset_size': 1000}}, 'simulation': {'name': 'IllustrisTNG', 'args': {'path': 'data/galaxy-id-12.hdf5'}}, 'output_path': 'output'}


2024-07-12 15:27:42,267 - rubix - WARNING - The Subset value is set in config. Using only subset of size 1000 for stars
2024-07-12 15:27:42,272 - rubix - WARNING - The Subset value is set in config. Using only subset of size 1000 for gas


['age', 'apply_subset', 'coords', 'mass', 'metallicity', 'velocity']
<class 'jaxlib.xla_extension.ArrayImpl'>


In [6]:
# NBVAL_SKIP
from rubix.utils import print_hdf5_file_structure

print(print_hdf5_file_structure("output/rubix_galaxy.h5"))

File: output/rubix_galaxy.h5
Group: galaxy
    Dataset: center (float64) ((3,))
    Dataset: halfmassrad_stars (float64) (())
    Dataset: redshift (float64) (())
Group: meta
    Dataset: BoxSize (float64) (())
    Dataset: CutoutID (int64) (())
    Dataset: CutoutRequest (object) (())
    Dataset: CutoutType (object) (())
    Dataset: Git_commit (|S40) (())
    Dataset: Git_date (|S29) (())
    Dataset: HubbleParam (float64) (())
    Dataset: MassTable (float64) ((6,))
    Dataset: NumFilesPerSnapshot (int64) (())
    Dataset: NumPart_ThisFile (int32) ((6,))
    Dataset: Omega0 (float64) (())
    Dataset: OmegaBaryon (float64) (())
    Dataset: OmegaLambda (float64) (())
    Dataset: Redshift (float64) (())
    Dataset: SimulationName (object) (())
    Dataset: SnapshotNumber (int64) (())
    Dataset: Time (float64) (())
    Dataset: UnitLength_in_cm (float64) (())
    Dataset: UnitMass_in_g (float64) (())
    Dataset: UnitVelocity_in_cm_per_s (float64) (())
Group: particles
    Group

In [7]:
# NBVAL_SKIP
from rubix.utils import load_galaxy_data

load_galaxy_data("output/rubix_galaxy.h5")

({'subhalo_center': array([11413.86337268, 35893.86374042, 32027.01940138]),
  'subhalo_halfmassrad_stars': 7.727193253526112,
  'redshift': 2.220446049250313e-16,
  'particle_data': {'gas': {'coords': array([[11414.05876352, 35893.6833483 , 32027.13166924],
           [11414.14656631, 35893.73356692, 32027.14557414],
           [11414.01824255, 35893.65232478, 32027.07648623],
           ...,
           [11409.61451254, 35888.5063995 , 32044.53073751],
           [11424.19250637, 35896.63240307, 32028.59975885],
           [11417.12077374, 35895.52565889, 32034.56892833]]),
    'density': array([3.6664732e+07, 2.1717842e+07, 5.0334548e+07, ..., 4.7079717e+03,
           9.9538037e+03, 1.1597330e+04], dtype=float32),
    'electron_abundance': array([1.1116004, 1.1071373, 1.0759586, ..., 1.2056972, 1.2106973,
           1.215307 ], dtype=float32),
    'internal_energy': array([1.1189402e+13, 9.2231906e+12, 9.2254587e+12, ..., 4.0579005e+14,
           5.7598484e+14, 1.7665127e+14], dtyp